# TensorFlow import

In [1]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.9.0


## Tensorflow Execution Style
* Eager execution
 * 변수값을 바로 확인할 수 있으며, 마치 numpy 짜듯이 짠다.
* Graph execution
 * Tensorflow의 초창기 구현 방법
 * Graph 구성과 실행(Session)하는 부분으로 분리가 된다.
  * Phase1: Graph 구성
  * Phase2: Session Run
 

# Graph 생성
* 변수를 그래프 안에다가 생성하고 그것에 대한 연산을 정의하는 단계이다.
* 이 단계에서는 계산을 하더라도 계산된 값이 나오지 않고 텐서가 나온다.

## Tensor란 무엇인가?
* An n-dimensional array이다.
 * 0-d tensor: scalar (number)
 * 1-d tensor: vector
 * 2-d tensor: matrix
....

<img src="https://github.com/modulabs/modu-tensorflow/raw/41c0e899d1b65075320adae03b2732734665d9fb/figures/TF_graph.png" width="40%">

In [2]:
test = tf.add(5,6) # 이 경우 자동으로 5와 6에 naming이 된다.

In [3]:
print(test)

Tensor("Add:0", shape=(), dtype=int32)


### 다음과 같은 표현을 한다.
Node: Operators, Variable, and Constant <br>
Edge: tensors.....<br>
Tensors 는 데이터이다.

<img src="https://github.com/modulabs/modu-tensorflow/raw/41c0e899d1b65075320adae03b2732734665d9fb/figures/TF_add1.png" width="40%">

# Session 실행하기
* 위의 과정은 단지 그래프의 형태만 만들어놓은 상태이다.
* 실제 그래프의 계산은 tf.Session()이라는 것을 실행하여 계산한다.
* 마치 파이프에 물이 흐르듯이 정의한 그래프의 연산을 수행한다.
* 그래서 Tensorflow는 tensor가 flow한다해서 Tensorflow!!
* tf.Session()을 열면 TF default로 GPU 메모리를 다 잡아버리므로
 * 이것을 방지하기 위해서 gpu_options를 준다.


### allow_growth 옵션이란?
* 기본적으로 TensorFlow는 GPU를 쓸 때 거의 대부분의 GPU 메모리를 할당하는데 이것은 디바이스 메모리 단편화를 방지해서 메모리를 효율적으로 쓰게 하는 것을 위함이다.
* 그런데 때로는 필요한 만큼의 메모리만 할당하는 것이 유용할 때가 있는 데 그것을 가능하게 하는 것이 allow_growth 옵션이다. 이러한 옵션으로 처음에는 매우 작은 메모리만 할당하다가 세션이 실행되면서 더 많은 GPU 메모리 할당이 필요해지면 Tensorflow는 필요한 메모리 영역을 증가시켜 추가로 할당한다.
* 옵션은 tf.ConfigProto로 설정한다.

https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/using_gpu/

<img src="https://github.com/modulabs/modu-tensorflow/raw/41c0e899d1b65075320adae03b2732734665d9fb/figures/TF_add2.png" width="40%">

In [4]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
a = tf.add(3,5)
# sess=tf.Session()
sess = tf.Session(config=sess_config)
print(sess.run(a))
sess.close() # session을 열면 sess.close()를 해서 명시적으로 닫아줘야 한다.

8


<img src="" width=>

# with 구문으로 처리해버리기
* with 구문을 사용하면 close하는 작업을 하지 않아도 된다.

In [5]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
a = tf.add(5,6)
# sess=tf.Session()
with tf.Session(config=sess_config) as sess:
    print(sess.run(a))
# close를 지알아서 해준다.

11


# 구문 안에서는 sess.run()대신 eval() method를 쓸 수 있다

In [6]:
with tf.Session(config=sess_config) as sess:
      print(a.eval())

11


# tf.InteractiveSession()
* with 구문이 없어도 그래프 실행시킬 수 있다.

In [7]:
sess = tf.InteractiveSession(config=sess_config)
print(a.eval())

11


# 조금 더 복잡한 그래프의 계산

In [8]:
x = 2
y = 3
w = tf.add(x, y)
z = tf.multiply(x, y)
p = tf.pow(z, w)
print(p)
with tf.Session(config=sess_config) as sess:
  print(sess.run(p))

Tensor("Pow:0", shape=(), dtype=int32)
7776


<img src="https://github.com/modulabs/modu-tensorflow/raw/41c0e899d1b65075320adae03b2732734665d9fb/figures/TF_more_calculation.png" width="40%">

# Subgraphs
* tf.Session.run(fetchs, feed_dict=None,options=None, run_metadata=None )

In [9]:
x = 2 
y = 3
add_op = tf.add(x,y) # 5
mul_op =tf.multiply(x,y) # 6
useless = tf.multiply(x, add_op) #10
pow_op = tf.pow(add_op, mul_op) #5^6
with tf.Session() as sess:
    z, not_useless = sess.run([pow_op, useless])
    print(z, not_useless)

15625 10


<img src="./img/sub.jpg" width="40%">

## 그래프를 여러개의 GPU, CPU로 쪼갤 수 있다.
ex) AlexNet 2개의 GPU로 학습
* 연산과 텐서가 어떤 디바이스에 배치되었는지 알아보기 위해, 세션을 만들 때 log_device_placement 옵션을 True로 설정한다
* tf.device('/원하는 장치:number')로 원하는 디바이스에 할당할 수 있다.


In [38]:
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
  c = tf.multiply(a, b)

# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Runs the op.
print(sess.run(c))


[ 1.  4.  9. 16. 25. 36.]


# 우리는 하나 이상의 그래프를 만들 수 있다.
그리고 기본으로는 default graph를 실행한다.

In [23]:
# 여기다가 뭔가의 그래프를 정의한다.
# 우리가 실행할 그래프 .... 무언가....
g = tf.Graph()
with g.as_default():
    x = tf.add(7, 8)

In [21]:
# 위에서 정의한 그래프를 실행한다.
with tf.Session(graph=g) as sess:
    print(sess.run(x))

15


In [25]:
g = tf.get_default_graph()

In [41]:
g1 = tf.get_default_graph()
g2 = tf.Graph()
# add ops to the default graph
with g1.as_default():
    a = tf.constant(3)
# add ops to the user created graph
with g2.as_default():
    b = tf.constant(5)

In [43]:
with tf.Session(graph=g1) as sess:
    print(sess.run(a))

3


In [42]:
with tf.Session(graph=g2) as sess:
    print(sess.run(b))

5
